# Intel® Neural Compressor Sample for Tensorflow

## Introduction

This is a demo to show an End-To-End pipeline to speed up AI model by Intel® Neural Compressor.

1. Train a CNN AlexNet model by Keras and Intel Optimization for Tensorflow based on dataset MNIST.

2. Quantize the frozen PB model file by Intel® Neural Compressor to INT8 model.

3. Compare the performance of FP32 and INT8 model by same script.


## Run in Intel® DevCloud

### Find a Computing Node

In [1]:
!echo "* How many compute servers are available?"
!pbsnodes | grep "^s" | wc -l

!echo "* How many of them are free?"
!pbsnodes | grep "state = free" | wc -l

!echo "* What are the time limits for queued jobs?"
!qmgr -c 'p q batch' | grep walltime

!echo "* What is the configuration of the available compute servers?"
!pbsnodes | grep properties | sort | uniq

* How many compute servers are available?
246
* How many of them are free?
176
* What are the time limits for queued jobs?
set queue batch resources_max.walltime = 24:00:00
set queue batch resources_default.walltime = 06:00:00
* What is the configuration of the available compute servers?
     properties = core,cfl,i9-10920x,ram32gb,net1gbe,gpu,iris_xe_max,dual_gpu
     properties = core,cfl,i9-10920x,ram32gb,net1gbe,gpu,iris_xe_max,quad_gpu
     properties = xeon,cfl,e-2176g,ram64gb,net1gbe,gpu,gen9
     properties = xeon,clx,ram192gb,net1gbe,fpga_runtime,fpga,stratix10
     properties = xeon,icx,gold6348,ramgb,netgbe,jupyter,batch
     properties = xeon,skl,gold6128,ram192gb,net1gbe,fpga_runtime,fpga,arria10
     properties = xeon,skl,gold6128,ram192gb,net1gbe,jupyter,batch
     properties = xeon,skl,gold6128,ram192gb,net1gbe,jupyter,batch,fpga_compile
     properties = xeon,skl,ram384gb,net1gbe,renderkit


In [2]:
!pbsnodes

s001-n001
     state = job-exclusive
     power_state = Running
     np = 2
     properties = xeon,skl,gold6128,ram192gb,net1gbe,jupyter,batch
     ntype = cluster
     jobs = 0/1838653.v-qsvr-1.aidevcloud,1/1838675.v-qsvr-1.aidevcloud
     status = rectime=1642556489,macaddr=a4:bf:01:38:e0:68,cpuclock=Fixed,varattr=,jobs=1838653.v-qsvr-1.aidevcloud(cput=17,energy_used=0,mem=181264kb,vmem=1356372kb,walltime=10248,session_id=1312338) 1838675.v-qsvr-1.aidevcloud(cput=6,energy_used=0,mem=110928kb,vmem=339264kb,walltime=7132,session_id=1321882),state=free,netload=2374544737202,gres=,loadave=0.01,ncpus=24,physmem=196735540kb,availmem=196166096kb,totmem=198734384kb,idletime=232988,nusers=4,nsessions=5,sessions=1029988 1030006 1312338 1313323 1321882,uname=Linux s001-n001 5.4.0-80-generic #90-Ubuntu SMP Fri Jul 9 22:49:44 UTC 2021 x86_64,opsys=linux
     mom_service_port = 15002
     mom_manager_port = 15003

s001-n045
     state = down
     power_state = Running
     np = 2
     properties =

In [2]:
!bash run_inc_ft_mnist_sample.sh

Train Model by Keras/Tensorflow with MNIST
/glob/intel-python/python3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Tensorflow version 1.13.1
Traceback (most recent call last):
  File "keras_tf_train_mnist.py", line 9, in <module>
    import neural_compressor as inc
ModuleNotFoundError: No module named 'neural_compressor'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "keras_tf_train_mnist.py", line 13, in <module>
    import lpot as inc
ModuleNotFoundError: No module named 'lpot'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "keras_tf_train_mnist.py", line 16, in <module>
    import ilit as inc
ModuleNotFoundError

In [14]:
!bash t.sh

ls: invalid option -- '3'
Try 'ls --help' for more information.
error


![fp32_int8_aboslute.png](fp32_int8_aboslute.png)

![fp32_int8_times.png](fp32_int8_times.png)